In [1]:
import os
import xml.etree.ElementTree as ET
import sys
import idyntree.bindings as iDynTree
from urdfModifiers.utils import *
__file__ = "/root/Github/human-model-generator/code/test.ipynb"
sys.path.append(os.path.join(os.path.dirname(__file__), "src"))
sys.path.append(os.getcwd())
from src import *
from config import *
import mujoco
import mujoco_py

In [2]:
urdf_file_path = "/root/Github/human-model-generator/code/models/humanModels/test.urdf"
#
write_urdf_with_changes(urdf_file_path) # not useful



In [12]:
# model_human = mujoco.MjModel.from_xml_path(urdf_file_path) # lack pelvis info
human_urdf, gazebo_plugin_text = utils.load_robot_and_gazebo_plugins(
    urdf_file_path, "no_gazebo_plugins.urdf") 
# mujoco.mj_saveLastXML("/root/Github/human-model-generator/code/models/humanModels/test.xml", model_human)
# remove_autolimits_attribute("/root/Github/human-model-generator/code/models/humanModels/mocap_v2_scaled.xml")
model_robot = mujoco.MjModel.from_xml_path("/root/Github/human-model-generator/code/models/humanModels/mocap_v2.xml")
mujoco.mj_saveLastXML("/root/Github/human-model-generator/code/models/humanModels/mocap_v2.xml", model_robot)
human_urdf.link_map.keys()

dict_keys(['world', 'Pelvis', 'LowerTrunk_f1', 'LowerTrunk', 'UpperTrunk_f1', 'UpperTrunk_f2', 'UpperTrunk', 'RightShoulder', 'LeftShoulder', 'Neck_f1', 'Neck_f2', 'Neck', 'Head_f1', 'Head', 'RightShoulder_f1', 'RightShoulder_f2', 'RightUpperArm', 'RightForeArm_f1', 'RightForeArm', 'RightHand_f1', 'RightHand', 'RightHandCOM', 'LeftShoulder_f1', 'LeftShoulder_f2', 'LeftUpperArm', 'LeftForeArm_f1', 'LeftForeArm', 'LeftHand_f1', 'LeftHand', 'LeftHandCOM', 'RightUpperLeg_f1', 'RightUpperLeg_f2', 'RightUpperLeg', 'RightLowerLeg_f1', 'RightLowerLeg', 'RightFoot_f1', 'RightFoot_f2', 'RightFoot', 'RightToe', 'RightFTSensor', 'LeftUpperLeg_f1', 'LeftUpperLeg_f2', 'LeftUpperLeg', 'LeftLowerLeg_f1', 'LeftLowerLeg', 'LeftFoot_f1', 'LeftFoot_f2', 'LeftFoot', 'LeftToe', 'LeftFTSensor', 'BicBrac_RUA', 'BicBrac_RFA', 'TricBrac_RUA', 'TricBrac_RFA', 'FlexCarp_RFA', 'FlexCarp_RH', 'ExtCarp_RFA', 'ExtCarp_RH', 'BicBrac_LUA', 'BicBrac_LFA', 'TricBrac_LUA', 'TricBrac_LFA', 'FlexCarp_LFA', 'FlexCarp_LH', 'E

In [15]:
model_human = mujoco.MjModel.from_xml_path("/root/Github/human-model-generator/code/models/humanModels/mocap_v2_scaled.xml")

74.34031250000005

In [4]:
# write_urdf_with_changes("/root/Github/human-model-generator/code/models/humanModels/base.urdf")
# model_test = mujoco.MjModel.from_xml_path("/root/Github/human-model-generator/code/models/humanModels/base.xml")
# mujoco.mj_saveLastXML("/root/Github/human-model-generator/code/models/humanModels/base.xml", model_test)
# human_urdf.link_map["LeftFoot"].inertial.mass
def get_valid_names(human_urdf):
    names = list(human_urdf.link_map.keys())
    names = [name for name in names if "_" not in name and "world" not in name]
    names = [name for name in names if human_urdf.link_map[name].inertial.mass > 1e-4]
    return names
urdf_names = get_valid_names(human_urdf)
urdf_masses = [human_urdf.link_map[name].inertial.mass for name in urdf_names]
sum(urdf_masses)
# urdf_names, urdf_masses
name_map = {
    "root": "Pelvis",
    "lfemur": "LeftUpperLeg",
    "ltibia": "LeftLowerLeg",
    "lfoot": ("LeftFoot", "LeftToe"),
    "rfemur": "RightUpperLeg",
    "rtibia": "RightLowerLeg",
    "rfoot": ("RightFoot", "RightToe"),
    "upperback": "UpperTrunk",
    "throax": "LowerTrunk",
    "lowerneck": ("Head", "Neck"),
    "lclavicle": "LeftShoulder",
    "lhumerus": "LeftUpperArm",
    "lradius": "LeftForeArm",
    "lwrist": "LeftHand",
    "rclavicle": "RightShoulder",
    "rhumerus": "RightUpperArm",
    "rradius": "RightForeArm",
    "rwrist": "RightHand"
}

mass_dict = {}
for k in name_map:
    if isinstance(name_map[k], tuple):
        mass_dict[k] = sum([human_urdf.link_map[name].inertial.mass for name in name_map[k]])
    else:
        mass_dict[k] = human_urdf.link_map[name_map[k]].inertial.mass

        
mass_dict, sum(mass_dict.values())

({'root': 8.8725,
  'lfemur': 10.8525,
  'ltibia': 3.4274999999999998,
  'lfoot': 0.9974999999999999,
  'rfemur': 10.8525,
  'rtibia': 3.4274999999999998,
  'rfoot': 0.9974999999999999,
  'upperback': 3.9225,
  'throax': 11.617500000000001,
  'lowerneck': 5.1075,
  'lclavicle': 3.9225,
  'lhumerus': 1.9725000000000001,
  'lradius': 1.125,
  'lwrist': 0.4425,
  'rclavicle': 3.9225,
  'rhumerus': 1.9725000000000001,
  'rradius': 1.125,
  'rwrist': 0.4425},
 74.99999999999999)

In [5]:
# model_robot.body_inertia.shape, model_human.body_inertia.shape
# model_robot.body_mass.shape, model_human.body_mass.shape
# model_human, model_robot

# Print all body names

for i in range(model_robot.nbody):
    print(f"Body {i}: {model_robot.body(i).name}, mass: {model_robot.body_mass[i]}")
print("sum of masses:", sum(model_robot.body_mass))


# body_id = model_human.body("Pelvis").id

Body 0: world, mass: 0.0
Body 1: root, mass: 5.575279762570686
Body 2: lhipjoint, mass: 0.0
Body 3: lfemur, mass: 3.5063369632240877
Body 4: ltibia, mass: 1.779256777646115
Body 5: lfoot, mass: 1.23264
Body 6: rhipjoint, mass: 0.0
Body 7: rfemur, mass: 3.5063369632240877
Body 8: rtibia, mass: 1.779256777646115
Body 9: rfoot, mass: 1.23264
Body 10: lowerback, mass: 0.0
Body 11: upperback, mass: 1.4367550402417326
Body 12: thorax, mass: 4.188790204786391
Body 13: lowerneck, mass: 2.1446605848506324
Body 14: upperneck, mass: 0.0
Body 15: head, mass: 0.0
Body 16: lclavicle, mass: 0.7365061842714081
Body 17: lhumerus, mass: 0.86717769480133
Body 18: lradius, mass: 0.41397028195740504
Body 19: lwrist, mass: 0.26808257310632905
Body 20: rclavicle, mass: 0.7365061842714081
Body 21: rhumerus, mass: 0.86717769480133
Body 22: rradius, mass: 0.41397028195740504
Body 23: rwrist, mass: 0.26808257310632905
sum of masses: 30.9534265424628


In [6]:

import mujoco
import numpy as np

# Usage
model_path = "/root/Github/human-model-generator/code/models/humanModels/mocap_v2.xml"
height_info = calculate_humanoid_height(model_path)

# Save the modified XML model file
print(f"total height: {height_info:.4f} meters")


total height: 1.4954 meters


In [11]:

real_height = 1.75 
scaled_model_path = "/root/Github/human-model-generator/code/models/humanModels/mocap_v2_scaled.xml"
# scale_humanoid_model(model_path, scaled_model_path, real_height, weight = 75)
scale_humanoid_model(model_path, scaled_model_path, real_height, weight = 75)



Scaling the model size to 1.75 meters
total mass: 75 kg
Saving the scaled model to /root/Github/human-model-generator/code/models/humanModels/mocap_v2_scaled.xml


In [8]:
import xml.etree.ElementTree as ET
# remove_autolimits_attribute(scaled_model_path)
model = mujoco.MjModel.from_xml_path(scaled_model_path)
print_body_info_table(model)

Body Information
Body Name          | Body Mass 
-------------------|-------------------------
Body 0: world               | 0.0                
Body 1: root                | 13.50887539443095  
Body 2: lhipjoint           | 0.0                
Body 3: lfemur              | 8.495837185619811  
Body 4: ltibia              | 4.311130405559332  
Body 5: lfoot               | 2.986680646589392  
Body 6: rhipjoint           | 0.0                
Body 7: rfemur              | 8.495837185619811  
Body 8: rtibia              | 4.311130405559332  
Body 9: rfoot               | 2.986680646589392  
Body 10: lowerback           | 0.0                
Body 11: upperback           | 3.4812503833882924 
Body 12: thorax              | 10.149418027371114 
Body 13: lowerneck           | 5.196502030014011  
Body 14: upperneck           | 0.0                
Body 15: head                | 0.0                
Body 16: lclavicle           | 1.784550855608138  
Body 17: lhumerus            | 2.101167281783110

SyntaxError: invalid syntax (1545386278.py, line 1)